# 1. Importing the data from csv file and plotting glucose readings along with meal times

In [ ]:
#This was written in Python 2, if you are running Python 3, use a code translation library

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, date, time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import math

%matplotlib inline
#This is graph line style, from Nate silver
plt.style.use('fivethirtyeight')

#_DATA_DIR = r'/Users/administrator/Desktop/New Git Diabetes/Input/input'
#_FIG_DIR = r'/Users/administrator/Desktop/New Git Diabetes/figures'
_DATA_DIR = r'C:\Users\engadmin\Desktop\Diabetes  Project Code\Input\input'
_FIG_DIR = r'C:\Users\engadmin\Desktop\Diabetes  Project Code\figures'

if not os.path.exists(_FIG_DIR):
    os.makedirs(_FIG_DIR)
    
_GULCOSE_DATA_PATH = os.path.join(_DATA_DIR, 'P1.csv')
fields = ["Date", "mg/dL", "CHO (g)", "Fat (g)", "Protein (g)"]
df_P1 = pd.read_csv(_GULCOSE_DATA_PATH, parse_dates=['Date'], usecols = fields) #read and prase Date col as dataTimes object
df_P1 = df_P1.sort(['Date'])

## Matplotlib Variables
_FIG_SIZE = (16, 8)
_FIG_FORMAT = 'png'
_FIG_DPI = 200

def _file_format(string_):
    string_ = string_.replace('-', '_').replace(' ', '_').replace('$', '')
    string_ += '.' + _FIG_FORMAT
    return string_

#For printing full elements of dataframe or array
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

print "\nTotal number of glucose readings is:", df_P1['mg/dL'].size
print "\nSome of points may be missing, the above number represents the collected readings from start to end date"

#Removing NaN from a specific column of the dataframe, here it's mg/dL
df_P1_wonan_mg = df_P1.dropna(subset=['mg/dL'])
print "\nRemove NaN from the mg/dL column...", "\nTotal actual number of glucose readings is:", df_P1_wonan_mg['mg/dL'].size

#Converting object type to float for all of the dataframe
df_P12= pd.DataFrame(df_P1_wonan_mg).convert_objects(convert_numeric=True)

print "\nRemove NaN from the CHO so we can plot the points of meals..."
df_P12 = df_P12.dropna(subset=['CHO (g)'])
print "Total number of meal times:", df_P12['CHO (g)'].size

print "\nSome of the successive meal readings are redundant and stand for one meal only"

df_P12 = df_P12.reset_index(drop=True)

df_P1_meals = copy.deepcopy(df_P12)

#print df_P1_meals['CHO (g)']

print '\nAdd succesive redundant meal readings (CHO, fat, protein) and store under one recording...\n' 
row = 0

count = 1
for i, x in enumerate(df_P12['CHO (g)']):
    if i > 0:
        row = row + 1
        if (i == (df_P12['CHO (g)'].size-1) and x == df_P12['CHO (g)'][i-1]):#checking if it's last element of the list
            count += 1
            df_P1_meals.drop(df_P1_meals.index[row], axis=0, inplace=True)
            df_P1_meals = df_P1_meals.reset_index(drop=True)
            df_P1_meals.set_value(row-1,'CHO (g)',df_P12['CHO (g)'][i-1]*count)
        
        #if content of current element is same as previous, then delete row      
        elif x == df_P12['CHO (g)'][i-1]:
            count += 1
            df_P1_meals.drop(df_P1_meals.index[row], axis=0, inplace=True)
            df_P1_meals = df_P1_meals.reset_index(drop=True)
            row = row - 1
            
        #if next meal reading is different than the previous one    
        else:
            if(i == (df_P12['CHO (g)'].size-1)):
                df_P1_meals.set_value(row-1,'CHO (g)',df_P12['CHO (g)'][i-1]*count)
                df_P1_meals.set_value(row,'CHO (g)',df_P12['CHO (g)'][i])
            else:
                df_P1_meals.set_value(row-1,'CHO (g)',df_P12['CHO (g)'][i-1]*count)
                #print "Current value is different than previous. Multiply previous by", count,"and result is:",df_P12['CHO (g)'][i-1]*count
                #print "this will be in the row index of:", row-1
                count = 1   
                
#print df_P1_meals  
                
df_P1_meals = df_P1_meals.reset_index(drop=True)
print "There are", df_P1_meals['CHO (g)'].size, " unredundant recorded glucose readings with meal times.\n" 

print df_P1_meals.head()

plt.figure(figsize=_FIG_SIZE)
plt.plot_date(df_P1.Date, df_P1['mg/dL'], '-', label="Glucose Level");
plt.plot_date(df_P1_meals.Date, df_P1_meals['mg/dL'], 'ro', label="Meal Starting Time")
plt.xlabel('Date')
plt.ylabel('Glucose Reading (mg/dl)')
plt.legend(loc='upper right')
title = 'Glucose Readings with the meal times'
plt.title(title);

# Patient meal size histogram & Stats

In [ ]:
bins =  int(math.ceil(np.max(df_P1_meals['CHO (g)'])/5))#bins of 5 grams

n, bins, patches = plt.hist(df_P1_meals['CHO (g)'], bins, facecolor='blue', alpha=1)
plt.title('Patient Meal Carbs Size Histogram')
plt.xlabel('Meal Carbs (CHO) in grams')
plt.ylabel('# of meals')
plt.show()

print "The meal Carbs mean is:", np.mean(df_P1_meals['CHO (g)']), "grams with a std of:", np.std(df_P1_meals['CHO (g)']), "grams."

# Interactive Plotting of the data

In [ ]:
#Interactive plotting using bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.models.markers import Asterisk
from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.models import HoverTool

# output to static HTML file
output_file("All_Data.html")

plot = figure(plot_width=1250, plot_height=600, title="Glucose Readings Interactive Plot",
              tools='pan,wheel_zoom,reset,box_zoom,save, undo', x_axis_type='datetime')

s1 = plot.line(df_P1.Date, df_P1['mg/dL'],  line_width=2, line_alpha=0.6)
s2 = plot.square(df_P1.Date, df_P1['mg/dL'], legend="Glucose Readings", fill_color=None, line_color="grey")

source = ColumnDataSource(dict(x=df_P1_meals.Date, y=df_P1_meals['mg/dL'], z=df_P1_meals['CHO (g)'], k = df_P1_meals['Fat (g)'], d = df_P1_meals['Protein (g)']))

glyph_1 = Asterisk(x="x", y="y", line_color="#f0027f", fill_color=None, line_width=8)
s3 = plot.add_glyph(source, glyph_1)

plot.add_tools(HoverTool(renderers=[s3], tooltips= [
    ('Glucose (mg/dL)', '@y'),
    ('Meal Size (grams)', '@z'),
    ('Fat (g)', '@k'),
    ('Protein (g)', '@d')
]))

plot.xaxis.axis_label = "Date & Time"
plot.yaxis.axis_label = "Glucose Reading (mg/dL)"

show(plot)

# Critical Assumptions for Cleaning Data

## 1. Assumption for Deletion

**a. Delete meal times that have several missing glucose readings and do not have a full glucose plot following them**

We may consider keeping these points later if we are interested on the meal times only and not the readings following them.

The window that is selected is under window_size. i.e: 17 represents a window of (17*5) + 5 = 90 mintues

In [ ]:
import datetime

df_P1_meals_w_deletion = copy.deepcopy(df_P1_meals)

#variables
incr = 5 #Data was recorded at 5 minutes increments
window_size = 17 #This represents window size based on increments of 5.

del_points_df1 = pd.DataFrame()#Dataframe containing meal to be deleted by this assumption
points_to_del1 = []

for x in range(df_P1_meals_w_deletion['CHO (g)'].size):
    a = df_P1_meals_w_deletion['Date'][x]
    b = a
    count = 0
    for j in range(window_size):
        b += datetime.timedelta(minutes=incr)
        reading = df_P1.loc[df_P1['Date'] == b]['mg/dL']
        try:
            reading.values[0]
        except IndexError:
            #print "Index out of range"
            count +=1 
            continue  
        if (math.isnan(reading.values[0])):
            count +=1
    if (count > (window_size/3)):
        print "\nThe meal time at", a, "has", count,"missing readings within its {}".format((window_size*5)+5),"minutes. Thus, it will be removed from the dataset.\n"
        #Put this into a dataframe to plot later with different color
        del_points_df1 = del_points_df1.append(df_P1.loc[df_P1['Date'] == a])        
        points_to_del1.extend([x])  

#Remove the above detected points from dataset
df_P1_meals_w_deletion.drop(df_P1_meals_w_deletion.index[points_to_del1],axis=0,inplace=True)
df_P1_meals_w_deletion = df_P1_meals_w_deletion.sort(['Date'])
df_P1_meals_w_deletion = df_P1_meals_w_deletion.reset_index(drop=True)

print "There are", df_P1_meals_w_deletion['CHO (g)'].size, "remaining meal times recordings.\n" 

#print points_to_del1

**The dataframe that contains the meal times to be deleted after this assumption is: del_points_df1**



**b. Delete meal times which all following readings are decreasing or have no sign of a significant increase (i.e: 15mg/dl).**

For every glucose reading starting few minutes before the meal was recorded, just in case the meal was recorded minutes later after food intake, we check if there is a significant increase in **its** following "glucose_effect_window" (i.e 45 minutes).

For instance, a reading situated 30 minutes after meal time recording which has a following glucose increase of (15mg/dl in total) after 30minutes will result on keeping that original meal time which was (30mins + 30mins before). Hence, this is a loose assumption but we can make it stricter by changing window variables and increase glucose value.

If no significant increase is apparent, we delete the meal point.


In [ ]:
#Note: Make sure to run the previous cell before running this one since the dataframe here used will be overwitten at end of this cell

#variables
incr = 5 #Data was collected at 5 minutes increments- Do not change this
glucose_effect_window = 45 #Minutes food may take to be reflected on the blood glucose
gl_increase_window = 30 #Minutes from the instance where the glucose is reflected on the blood to significant increase###Change this to higher minutes if glucose increases slowly
pre_recording_mins = 10 #Minutes that may have elapsed before patient has recorded the reading
gl_increase = 20 #20mg/dL increase

########May want to change VARIABLES above for different patients, especially if food takes a while to affect their glucose level############
meal_affect_window = [i for i in np.arange(-(pre_recording_mins), glucose_effect_window+incr, incr)]

compare_window = [j for j in np.arange(incr, gl_increase_window+incr, incr)]
################################################################

#print meal_affect_window
#print compare_window

points_to_del = []
del_points_df = pd.DataFrame()#Dataframe containing meal to be deleted by this assumption
low_surrounding_gl = pd.DataFrame()#Dataframe containing points where the glucose will go to take a significant increase, not necessarily the very lowest points
offset_low_point_to_meal = []

for x, item in enumerate(df_P1_meals_w_deletion['CHO (g)']):
    
    a = df_P1_meals_w_deletion['Date'][x]   
    meal_reading = df_P1.loc[df_P1['Date'] == a]['mg/dL']
    b = a - datetime.timedelta(minutes=pre_recording_mins)
    
    keep = 0
    for i in meal_affect_window:
        current_reading = df_P1.loc[df_P1['Date'] == b]['mg/dL']
        try:
            current_reading.values[0]
        except IndexError:
            #print "Index out of range"
            b += datetime.timedelta(minutes=incr)   
            continue
            
        if (math.isnan(current_reading.values[0])):
            b += datetime.timedelta(minutes=incr)  
            continue            
        c = b
        for k in compare_window:#comparing the point with its following readings in compare window size
            c += datetime.timedelta(minutes=incr)
            compare_reading = df_P1.loc[df_P1['Date'] == c]['mg/dL']
            if ((compare_reading.values[0] - current_reading.values[0]) >gl_increase):
                #print "We should keep this reading"
                #Store this reading on a dataframe of low readings surrounding meals
                low_surrounding_gl = low_surrounding_gl.append(df_P1.loc[df_P1['Date'] == b])
                #print "a is:", a, "and b is:", b, "and c is:", c, "and difference between b and c is:", (c - b)
                if (b<=a):
                    offset_low_point_to_meal.extend([int(-(a-b).seconds / 60)])
                    #print "The time elapsed between original meal time and surrounding low point is:", "-", (a-b).seconds / 60 ,"minutes."        
                else:
                    offset_low_point_to_meal.extend([int((b-a).seconds / 60)])
                    #print "The time elapsed between original meal time and surrounding low point is:", "+", (b-a).seconds / 60 ,"minutes."  
                #offset_low_point_to_meal.extend([offset_seconds])
                keep = 1  
                break
                
        if (keep == 1):
            break
        b += datetime.timedelta(minutes=incr)    
        
        if (i == meal_affect_window[-1]):
            #print "The meal reading taken at" ,a, "should be deleted."
            #Put this into an dataframe to plot with different color
            del_points_df = del_points_df.append(df_P1.loc[df_P1['Date'] == a])
            #Also remove this from dataset   
            points_to_del.extend([x])                     

#print "\nThe dataframe of the points to be deleted is:\n", del_points_df 

#print (points_to_del)

print "\nThere are", len(points_to_del), "meal recordings that will be deleted."

df_P1_meals_w_deletion.drop(df_P1_meals_w_deletion.index[points_to_del],axis=0,inplace=True)
df_P1_meals_w_deletion = df_P1_meals_w_deletion.sort(['Date'])
df_P1_meals_w_deletion = df_P1_meals_w_deletion.reset_index(drop=True)

print "\nThere are", df_P1_meals_w_deletion['CHO (g)'].size, "remaining meal times recordings." 

low_surrounding_gl = low_surrounding_gl.sort(['Date'])
low_surrounding_gl = low_surrounding_gl.reset_index(drop=True)

#print "The difference between each recorded meal and surrounding point which has a following significant increase is (in minutes):\n"
#print offset_low_point_to_meal
#It is mostly (-20minutes) because we start checking 20 minutes previous to the recorded meal

**The dataframe that contains the meal times to be deleted after this assumption is: del_points_df**

**In the below interactive Bokeh plotting: Plotting all the meal points along with the ones to be deleted with different colors.**


In [ ]:
def handle_case():
    print "This dataframe does not have missing glucose readings following any meal time"

# output to static HTML file
output_file("All_Data_with_Deletion.html")

plot = figure(plot_width=1250, plot_height=600, title="Glucose Readings Interactive Plot",
              tools='pan,wheel_zoom,reset,box_zoom,save, undo', x_axis_type='datetime')

plot.line(df_P1.Date, df_P1['mg/dL'],  line_width=2, line_alpha=0.6)
plot.square(df_P1.Date, df_P1['mg/dL'], legend="Glucose Readings", fill_color=None, line_color="grey")

good_readings = ColumnDataSource(dict(x=df_P1_meals_w_deletion.Date, y=df_P1_meals_w_deletion['mg/dL'], z=df_P1_meals_w_deletion['CHO (g)'], k = df_P1_meals_w_deletion['Fat (g)'], d = df_P1_meals_w_deletion['Protein (g)']))
glyph_1 = Asterisk(x="x", y="y", line_color="#f0027f", fill_color=None, line_width=8)
g1 = plot.add_glyph(good_readings, glyph_1)

if (len(points_to_del1) == 0):
    handle_case()
else: 
    delete_readings_1a = ColumnDataSource(dict(x=del_points_df1.Date, y=del_points_df1['mg/dL'], z=del_points_df1['CHO (g)'], k = del_points_df1['Fat (g)'], d = del_points_df1['Protein (g)']))
    glyph_2 = Asterisk(x="x", y="y", line_color="#32CD32", fill_color=None, line_width=8)
    g2 = plot.add_glyph(delete_readings_1a, glyph_2)

if (len(points_to_del) == 0):
    handle_case()
else:        
     delete_readings_1b = ColumnDataSource(dict(x=del_points_df.Date, y=del_points_df['mg/dL'], z=del_points_df['CHO (g)'], k = del_points_df['Fat (g)'], d = del_points_df['Protein (g)']))
     glyph_3 = Asterisk(x="x", y="y", line_color="#0404B4", fill_color=None, line_width=8)
     g3 = plot.add_glyph(delete_readings_1b, glyph_3)

plot.asterisk(legend = 'Meal Recordings', line_color="#f0027f", fill_color=None, line_width=8)
plot.asterisk(legend = 'To be deleted after Assumption 1.a', line_color="#32CD32", fill_color=None, line_width=8)
plot.asterisk(legend = 'To be deleted after Assumption 1.b', line_color="#0404B4", fill_color=None, line_width=8)

#print y
plot.add_tools(HoverTool(renderers=[g1, g2,g3], tooltips= [
    ('Glucose (mg/dL)', '@y'),
    ('CHO (g)', '@z'),
    ('Fat (g)', '@k'),
    ('Protein (g)', '@d')
]))

plot.xaxis.axis_label = "Date & Time"
plot.yaxis.axis_label = "Glucose Reading (mg/dL)"

show(plot) 

**This completes the assumptions for deleting abnormal meal points**

## 2. Assumption for Adjusting Meal times: Finding surrounding lowest glucose levels

Extract lowest glucose reading surrounding the meal time, this is usually the point of glucose increase effect after meal. The window can be a variable, we are using [-20mins to +45mins] of the meal time.

By extracting these points, we can estimate that the true meal times happened few minutes right before these points (usually 0 mins to 30 mins before) -- This latter assumption (of this sentence) is not used anywhere but may be used for plotting.

**Finding the lowest glucose level surrounding the recorded meal time**

1. Find a significant blood glucose increase surrounding the recorded meal time, say 15mg/dl. / Already done in previous deletion task

2. Find the lowest point before that increase. That's what we will consider in our future processing. / Already done in previous deletion task

3. Need to find surrounding lowest point next to low point previously found. Since the low point previously found can be 20mins before meal, we start search here from that point and forward.

**Below is the dataframe that contains low glucose levels surrounding meal times, which have a significant following glucose increase.** Not only the surrounding lowest readings, but it must have a **nearby** glucose increase to it.

In [ ]:
gl_rise_window = [j for j in np.arange(0, gl_increase_window+incr, incr)]

offset_lowest_point_to_meal = []
offset_highest_point_to_meal = []
points_to_readjust = []

readjust_points_df = pd.DataFrame()
surrounding_lowest_points_df = pd.DataFrame()

for x,a in enumerate(low_surrounding_gl['Date']):
    b = a
    min_reading = df_P1.loc[df_P1['Date'] == a]['mg/dL']
    max_reading = df_P1.loc[df_P1['Date'] == a]['mg/dL']
    lowest_pt_index = 0
    highest_pt_index = 0

    for i,y in enumerate(gl_rise_window):
        #get the lowest point within the full window
        current_reading = df_P1.loc[df_P1['Date'] == b]['mg/dL'].values[0]
        try:
            current_reading
        except IndexError:
            print "Index out of range"
            b += datetime.timedelta(minutes=incr)   
            continue
        
        if (math.isnan(current_reading)):
            b += datetime.timedelta(minutes=incr)  
            continue
            
        #print "The current lowest reading is:", min_reading.values[0], "and compared reading is:", current_reading, "and y is:", y
        if (current_reading<min_reading.values[0]):  
            min_reading = df_P1.loc[df_P1['Date'] == b]['mg/dL'] 
            lowest_pt_index = y 
        elif (current_reading>max_reading.values[0]):  
            max_reading = df_P1.loc[df_P1['Date'] == b]['mg/dL'] 
            highest_pt_index = y  
        b += datetime.timedelta(minutes=incr) 
    #print x,": At date", a, "the lowest point is", min_reading.values[0], "of index", lowest_pt_index, "and the highest point is", max_reading.values[0], "of index",highest_pt_index 
    offset_lowest_point_to_meal.extend([lowest_pt_index])
    offset_highest_point_to_meal.extend([highest_pt_index])

    surrounding_lowest_points_df = surrounding_lowest_points_df.append(df_P1.loc[df_P1['Date'] == (a + datetime.timedelta(minutes=int(lowest_pt_index)))]) 

surrounding_lowest_points_df = surrounding_lowest_points_df.sort(['Date'])
surrounding_lowest_points_df = surrounding_lowest_points_df.reset_index(drop=True)

#The real time difference between the lowest point and meal time is the addition of difference between meal time and low surrounding point + between low surrounding point and lowest surrounding point
        
offset_meal2increase = np.add(offset_lowest_point_to_meal , offset_low_point_to_meal)    
    
#print offset_meal2increase

print "\nThe mean of the time elapsed between recorded meal time and convenient surrounding lowest point is %.2f" %np.mean(offset_meal2increase), "minutes \nwith a std of %.2f" %np.std(offset_meal2increase), "minutes."

#print "\nThe mean of the time elapsed difference between recorded meal time and surrounding peak point is %.2f" %np.mean(offset_highest_point_to_meal), "minutes \nwith a std of %.2f" %np.std(offset_lowest_point_to_meal), "minutes."

min_minutes = np.absolute(np.min(offset_meal2increase))
max_minutes = np.max(offset_meal2increase)

n, bins, patches = plt.hist(offset_meal2increase, (min_minutes+max_minutes)/5, facecolor='blue', alpha=1)
plt.title('Minutes between recorded meal time and lowest point with glucose increase')
plt.xlabel('Minutes')
plt.ylabel('# of meals')
plt.show()

**The dataframe that contains the meal times after deletion of abnormal meal recordings is: df_P1_meals_w_deletion**

**The dataframe that contains the lowest surrounding points with significant glucose increase is: surrounding_lowest_points_df**

# Detecting peaks following meal times

Steps:

1. Detect a significant glucose decrease

2. Find the highest glucose point up to that decrease

Some peaks may be before the recorded meal if for instance the increase 20mins before the recording is over 20mg/dl. These peaks will be re-examined for a more appropriate lowest point and peak.


In [ ]:
pointofincrease_df = copy.deepcopy(surrounding_lowest_points_df)

gl_decrease = 15 #15mg/dl of glucose drop (we noticed from the data that 10mg/dl would be too low and may result in not finding the highest peak after meal)

minutes2peak = 60#Minutes to reach the peak after meal
gl_decrease_window = 60 #Minutes it takes for a significant glucose decrease after meal

up2peak_window = [j for j in np.arange(incr, minutes2peak+incr, incr)]
compare_window = [j for j in np.arange(incr, gl_decrease_window+incr, incr)]

#print up2peak_window
#print compare_window

afterpeak_lowgl_df = pd.DataFrame()#This dataframe contains the points after the glucose drop following the peak
lowpoint_to_del_df = pd.DataFrame()#Dataframe contaning any meal recording to be deleted because they do not have a significant nearby glucose drop after the increase
peak_points_df = pd.DataFrame()#Dataframe containing peak points following meal times
lowpoint_to_del = []

#print "pointofincrease_df size is:", pointofincrease_df['CHO (g)'].size

for x, item in enumerate(pointofincrease_df['mg/dL']):   
    a = pointofincrease_df['Date'][x]   
    lowest_point_reading = df_P1.loc[df_P1['Date'] == a]['mg/dL']
    keep = 0
    b= a
    #print "---------------------------------Meal time date is:", a
    for i in up2peak_window:
        b += datetime.timedelta(minutes=incr)
        current_reading = df_P1.loc[df_P1['Date'] == b]['mg/dL']
        #print "**Following reading:", current_reading.values[0], "at the date of", b          
        c = b
        for k in compare_window:#comparing the point with its following readings in compare window size
            c += datetime.timedelta(minutes=incr)
            compare_reading = df_P1.loc[df_P1['Date'] == c]['mg/dL']
            #print "***Compared reading reading:", compare_reading.values[0], " at the date of", c  
            if ((current_reading.values[0] - compare_reading.values[0]) > gl_decrease):#significant glucose drop has been detected
                #Store this reading on a dataframe of points detecting glucose drop after peak
                #print "found it"
                afterpeak_lowgl_df = afterpeak_lowgl_df.append(df_P1.loc[df_P1['Date'] == c])
                keep = 1
                break
                
        if (keep == 1):
            break   
        
        if (i == up2peak_window[-1]):
            print "Could not detect a significant glucose drop after this meal recording", df_P1_meals_w_deletion['Date'][x], ". It is suggested to delete it"  
            lowpoint_to_del.extend([x])  
            lowpoint_to_del_df = lowpoint_to_del_df.append(df_P1.loc[df_P1['Date'] == a])

#Drop x from the "all the meals" dataframe and lowest surrounding dataframe as well
df_P1_meals_w_deletion.drop(df_P1_meals_w_deletion.index[lowpoint_to_del],axis=0,inplace=True)  
df_P1_meals_w_deletion = df_P1_meals_w_deletion.reset_index(drop=True)
 

surrounding_lowest_points_df.drop(surrounding_lowest_points_df.index[lowpoint_to_del],axis=0,inplace=True)  
surrounding_lowest_points_df = surrounding_lowest_points_df.reset_index(drop=True)

offset_meal2increase = [item for index, item in enumerate(offset_meal2increase) if index not in lowpoint_to_del] 
#print "offset_meal2increase size is:", len(offset_meal2increase)

afterpeak_lowgl_df = afterpeak_lowgl_df.reset_index(drop=True)
lowpoint_to_del_df = lowpoint_to_del_df.reset_index(drop=True)

# print "afterpeak_lowgl_df size is:", afterpeak_lowgl_df['CHO (g)'].size
# print "pointofincrease_df size is:", pointofincrease_df['CHO (g)'].size

#print "\nThe dataframe of the points after detecting a glucose drop:\n", afterpeak_lowgl_df 
def drange(start, stop, step):
    while start <= stop:
            yield start
            start += step
            
#print df_P1.iloc[-2]['Date']
    
for x, item in enumerate(afterpeak_lowgl_df['mg/dL']):
    meal_recording = df_P1_meals_w_deletion['Date'][x]
    #print "x is:", x , "and its date is:", afterpeak_lowgl_df['Date'][x]   
    a = surrounding_lowest_points_df['Date'][x]   
    current_reading = df_P1.loc[df_P1['Date'] == a]['mg/dL'].values[0]
    lowpt_after_peak = afterpeak_lowgl_df['Date'][x]   
    b= a
    #difference in minutes between lowest point and the point after glucose drop following peak
    window_peak = (lowpt_after_peak-a).seconds / 60
    #print window_peak
    peak_index = 0

    for i in drange(5,window_peak,5):        
        b += datetime.timedelta(minutes=incr)
        if (b <= df_P1.iloc[-2]['Date']):
            incrt_reading = df_P1.loc[df_P1['Date'] == b]['mg/dL'].values[0]   

            if (math.isnan(incrt_reading)):
                b += datetime.timedelta(minutes=incr)  
                continue

            if (incrt_reading > current_reading):
                current_reading = incrt_reading
                peak_index = i
            
    c = a + datetime.timedelta(minutes=int(peak_index))
    
    
    #if the peak is before the meal reading then reconsider the peak and change to the one after meal recording
    if (c<meal_recording):
        c = meal_recording
        lowest_pt_index = 0
        min_reading = df_P1.loc[df_P1['Date'] == meal_recording]['mg/dL'].values[0]
        #check for lowest point after meal_recording within 45mins
        for k in drange(5,glucose_effect_window,5):#comparing the point with its following readings in compare window size
            c += datetime.timedelta(minutes=incr)
            if (c > df_P1.iloc[-2]['Date']):
                continue
            else:
                compare_reading = df_P1.loc[df_P1['Date'] == c]['mg/dL'].values[0]
                if (compare_reading<min_reading):  
                    min_reading = compare_reading
                    lowest_pt_index = k
        #Replace lowest reading by this new correct one, in the surrounding_lowest_points_df dataframe        
        new_lowestpt = meal_recording + datetime.timedelta(minutes=int(lowest_pt_index))
        print "Replace the reading at time", a, "with the more appropriate lowest reading at", new_lowestpt

        surrounding_lowest_points_df.drop(surrounding_lowest_points_df.index[x],axis=0,inplace=True) 
        surrounding_lowest_points_df = surrounding_lowest_points_df.append(df_P1.loc[df_P1['Date'] == new_lowestpt])           
        surrounding_lowest_points_df = surrounding_lowest_points_df.sort(['Date'])
        surrounding_lowest_points_df = surrounding_lowest_points_df.reset_index(drop=True)    
        
        offset_meal2increase[x] = lowest_pt_index
        
        
        peak = df_P1.loc[df_P1['Date'] == new_lowestpt]['mg/dL'].values[0]
        start_time = new_lowestpt
        inctime = start_time
        peak_index = 0
        #append the peak point following the new lowest reading
        for k in drange(5,60,5):#comparing the point with its following readings in compare window size (60minutes)
            inctime += datetime.timedelta(minutes=incr)
            if (inctime > df_P1.iloc[-2]['Date']):
                continue
            else:        
                compare_reading = df_P1.loc[df_P1['Date'] == inctime]['mg/dL'].values[0]
                if (compare_reading>peak):  
                    peak = compare_reading
                    peak_index = k
                    
        peak_points_df = peak_points_df.append(df_P1.loc[df_P1['Date'] == (start_time+datetime.timedelta(minutes=int(peak_index)))])           
   
    else:
        peak_points_df = peak_points_df.append(df_P1.loc[df_P1['Date'] == c])

surrounding_lowest_points_df = surrounding_lowest_points_df.reset_index(drop=True) 
        
peak_points_df = peak_points_df.reset_index(drop=True)    

#print_full(surrounding_lowest_points_df)

#print peak_points_df 
peak_elapsed_time = []

#Need to make list with elapsed time between peak and meal recording for plotting, in minutes
for x, item in enumerate(afterpeak_lowgl_df['mg/dL']): 
    meal_recording = df_P1_meals_w_deletion['Date'][x]
    peak_reading = peak_points_df['Date'][x]      
    if (peak_reading>=meal_recording):
        topeakduration = (peak_reading-meal_recording).seconds / 60
    else:
        topeakduration = -(meal_recording-peak_reading).seconds / 60       
    peak_elapsed_time.append(int(topeakduration))
    
print "There are", surrounding_lowest_points_df['Date'].size, "remaining readings."


What a two recorded meals are close to each other, only few minutes apart. Usually, the first would still be affecting the glucose increase and the second will add more to that increase. Thus, we will combine the first and second into one and considered it as if it was recorded at the first recording. This is only if they are 60 minutes or less apart (The 60 minutes can be a variable).

1. Detect the peak points of the same date.

2. Add the meals of those dates.

3. Keep the first one and delete rest.

In [ ]:
remove_index = peak_points_df['Date'].size #setting this to some value it will never be so we can overwrite it later
remove_pts = []
for i in range(peak_points_df['Date'].size):
    if ((i < peak_points_df['Date'].size -1) and (peak_points_df['Date'][i] == peak_points_df['Date'][i+1]) and (i != remove_index)):
        print "There are two succesive meal readings which have the same peak.", "The dates of the two readings are:", df_P1_meals_w_deletion['Date'][i], "and", df_P1_meals_w_deletion['Date'][i+1]
        total_mealCHO = df_P1_meals_w_deletion['CHO (g)'][i] + df_P1_meals_w_deletion['CHO (g)'][i+1]
        total_mealfat = df_P1_meals_w_deletion['Fat (g)'][i] + df_P1_meals_w_deletion['Fat (g)'][i+1]
        total_mealProtein = df_P1_meals_w_deletion['Protein (g)'][i] + df_P1_meals_w_deletion['Protein (g)'][i+1]
        
        #Need to replace main dataframe containing the meal times, replace the row with the new meal sizes
        df_P1_meals_w_deletion.set_value(i, 'CHO (g)', total_mealCHO)
        df_P1_meals_w_deletion.set_value(i, 'Fat (g)', total_mealfat)
        df_P1_meals_w_deletion.set_value(i, 'Protein (g)', total_mealProtein)
        
        remove_index = i+1
        remove_pts.append(remove_index)
#remove the next rows with the same peak points from all the dataframes
df_P1_meals_w_deletion.drop(df_P1_meals_w_deletion.index[remove_pts], axis=0,inplace=True)
df_P1_meals_w_deletion = df_P1_meals_w_deletion.sort(['Date'])
df_P1_meals_w_deletion = df_P1_meals_w_deletion.reset_index(drop=True)

surrounding_lowest_points_df.drop(surrounding_lowest_points_df.index[remove_pts], axis=0,inplace=True)
surrounding_lowest_points_df = surrounding_lowest_points_df.sort(['Date'])
surrounding_lowest_points_df = surrounding_lowest_points_df.reset_index(drop=True)

peak_points_df.drop(peak_points_df.index[remove_pts], axis=0,inplace=True)
peak_points_df = peak_points_df.sort(['Date'])
peak_points_df = peak_points_df.reset_index(drop=True)

print "remove points list is:", remove_pts

print "offset_meal2increase is", offset_meal2increase, "of the size", len(offset_meal2increase)

for e, d in enumerate(offset_meal2increase):
    print "index is:", e, "and content is", d

    #when we delete we change index so change this
for i in range(len(remove_pts)):
    print "i is:", i
    index = remove_pts[i]
    print "offset_meal2increase[index]", offset_meal2increase[index-i]
    del offset_meal2increase[index-i]
    del peak_elapsed_time[index-i]

print "There are ", df_P1_meals_w_deletion['Date'].size, "meals remaining."


# Plotting: Actual meal time recordings & Surrounding lowest points.

1. Plot until next meal only

2. Plot until there is a significant increase of glucose

Finding the points where the glucose starts to increase after the peak, so we stop the plot at that point

In [ ]:
nonempty_readings = df_P1[pd.notnull(df_P1['Date'])]
last_row_date = nonempty_readings.iloc[-1, nonempty_readings.columns.get_loc('Date')]
#print last_row_date

gl_2nd_increase = 15 #second increase of 10mg/dl
after_reading = 300 #size of the plot after meal time in minutes
stop_plot_date = []

#print "surrounding_lowest_points_df size is ", surrounding_lowest_points_df['Date'].size

for x in range(surrounding_lowest_points_df['Date'].size):
    #print "Elapsed time from meal to peak is", peak_elapsed_time[x]
    stop = 0
    peak_gl_Date = peak_points_df['Date'][x]
    for d in drange(5,(after_reading-peak_elapsed_time[x]), 5):
     
        #current glucose level
        peak_gl_Date += datetime.timedelta(minutes=incr) 
        if ((peak_gl_Date - last_row_date).days >= 0 and (peak_gl_Date - last_row_date).seconds > 0):
            break
        
        current_gl_Date = peak_gl_Date
        compared_gl_date = peak_gl_Date
        current_gl = df_P1.loc[df_P1['Date'] == current_gl_Date]['mg/dL'].values[0] 
        
        for l in drange(5, gl_increase_window, 5):
            compared_gl_date += datetime.timedelta(minutes=incr)
            #print "compared_gl_date is", compared_gl_date, "and the difference is in days", (compared_gl_date - last_row_date).days, "and in seconds",(compared_gl_date - last_row_date).seconds 
            if ((compared_gl_date - last_row_date).days >= 0 and (compared_gl_date - last_row_date).seconds > 0):
                #print "Reached end"
                break
            else:    
                compared_gl = df_P1.loc[df_P1['Date'] == compared_gl_date]['mg/dL'].values[0]  
###            
            if (compared_gl - current_gl > gl_2nd_increase):
                #Find lowest point between current_gl and compared_gl
                lowest2nd_pt = current_gl
                lowest2nd_pt_date = current_gl_Date
                current_gl_temp_date = current_gl_Date
                for s in drange (5,(compared_gl_date - current_gl_Date).seconds / 60,5):
                    current_gl_temp_date += datetime.timedelta(minutes=incr)
                    current_gl_temp = df_P1.loc[df_P1['Date'] == current_gl_temp_date]['mg/dL'].values[0]  
                    if (current_gl_temp < lowest2nd_pt):
                        lowest2nd_pt = current_gl_temp
                        lowest2nd_pt_date = current_gl_temp_date
                stop = 1
                #Need to stop the plot at this current_gl date, so let's save it
                break
        if (stop == 1):
            #print "Stop here", current_gl_Date
            stop_plot_date.append(lowest2nd_pt_date)
            break
        if (d == after_reading-peak_elapsed_time[x]):
            #print "All the way"
            stop_plot_date.append(current_gl_Date)

#print stop_plot_date[1]    

In [ ]:
print "the length of the stop_plot_date is", len(stop_plot_date)

print "the size of meals is", df_P1_meals_w_deletion['CHO (g)'].size

print "the size of surrounding_lowest_points_df is", surrounding_lowest_points_df['CHO (g)'].size



Plot_window = [i for i in np.arange(-(pre_recording_mins), after_reading+incr, incr)]

#This includes all steps for correct plotting
def Plot_extraction_v2(df_P1_w_all_deletion, df_all_plots_v2):
    for x in range(df_P1_w_all_deletion['CHO (g)'].size):
        #save the readings surrounding every meal time in a new dataframe
        a = df_P1_w_all_deletion['Date'][x]
        
        #Check if it's last reading
        if (x<((df_P1_w_all_deletion['CHO (g)'].size))-1):
            a1 = df_P1_w_all_deletion['Date'][x+1]          
            
            #c is the date from the dataframe that contains where glucose starts increasing again AFTER the peak
            
            
            b = a - datetime.timedelta(minutes=pre_recording_mins)
            reading_list = []
            for j in Plot_window:
                reading = df_P1.loc[df_P1['Date'] == b]['mg/dL']
                c = stop_plot_date[x]  #Stop at this date because there is another glucose increase right after it
                try:
                    reading.values[0]
                except IndexError:
                    #print "Index out of range"
                    reading_list.append(float('nan'))
                    b += datetime.timedelta(minutes=incr)   
                    continue
                #Check to include points only up to next meal time, else put nan on that array so they are not plotted
                if ((a1 - b).total_seconds() < 0 ):#readings_distance = (a1 -a).total_seconds()/60 #The elapsed time in minutes 
                    #print "Elapsed time on this iteration is less than the full window time", x
                    reading_list.append(float('nan'))
                    b += datetime.timedelta(minutes=incr)
                    continue
                elif((c - b).total_seconds() < 0 ):    
                    reading_list.append(float('nan'))
                    b += datetime.timedelta(minutes=incr)
                    continue                  
                else:
                    #store this value on the jth column of xth row of the new dataframe
                    reading_list.append(reading.values[0])
                    b += datetime.timedelta(minutes=incr)
        else: 
            #plot readings until next meal, need to replace that row
            b = a - datetime.timedelta(minutes=pre_recording_mins)
            reading_list = []
            for j in Plot_window:
                reading = df_P1.loc[df_P1['Date'] == b]['mg/dL']       
                try:
                    reading.values[0]
                except IndexError:
                    #print "Index out of range"
                    reading_list.append(float('nan'))
                    b += datetime.timedelta(minutes=incr)   
                    continue
                #store this value on the jth column of xth row of the new dataframe
                reading_list.append(reading.values[0])
                b += datetime.timedelta(minutes=incr)
        #print reading_list
        #print df_all_plots.shape
        df_all_plots_v2.loc[x]=reading_list

    #print df_all_plots_v2

In [ ]:
def plotting(df_all_plots_v2, df_P1_w_all_deletion):
   
    plt.figure(figsize=(20,12))
    for j in range(10,20):
    #for j in range(len(df_P1_w_all_deletion)):
        row = df_all_plots_v2.iloc[j]
        row.plot()
        #row.plot(label='Meal Time at {}'.format(df_P1_w_all_deletion['Date'][j]))
        #Recorded meal time reading
        plt.plot(0, df_all_plots_v2[0][j], 'ro', markersize=10)
        #Lowest surrounding point of increase
        plt.plot(offset_meal2increase[j], surrounding_lowest_points_df['mg/dL'][j], 'b^', markersize=15)
        plt.plot(peak_elapsed_time[j], peak_points_df['mg/dL'][j], 'y*', markersize=20)      
        
    plt.xlabel('Minutes')
    plt.ylabel('Glucose Reading (mg/dl)')
    plt.legend(loc='upper right')
    title = 'Glucose Lelvel Readings {} minutes after meal recording or until next meal recording'.format(after_reading)
    plt.title(title);

    #Uncomment below to save plot
    #plt.savefig('Patient_1_Plotsaftermeal_iter0to5.png')

In [ ]:
df_all_plots_v2 = pd.DataFrame(columns=Plot_window)

Plot_extraction_v2(df_P1_meals_w_deletion, df_all_plots_v2)
plotting(df_all_plots_v2, df_P1_meals_w_deletion)

#print_full(df_P1_meals_w_deletion)

# Meal size Histogram after Data Cleaning

In [ ]:
bins =  int(math.ceil(np.max(df_P1_meals_w_deletion['CHO (g)'])/5))#bins of 5 grams

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
# rectangular box plot
bplot1 = axes[0].boxplot([df_P1_meals_w_deletion['CHO (g)'],df_P1_meals_w_deletion['Fat (g)'], df_P1_meals_w_deletion['Protein (g)']],
                         vert=True,   # vertical box aligmnent
                         patch_artist=True)

bplot0 = axes[1].hist(df_P1_meals_w_deletion['CHO (g)'], bins, facecolor='blue', alpha=1)
axes[1].set_xlabel('Meal Carbs (CHO) in grams')
axes[1].set_ylabel('# of meals')
axes[1].set_title('Patient Meal Carbs Size Histogram')

# fill with colors
colors = ['pink', 'lightblue', 'lightgreen']
for patch, color in zip(bplot1['boxes'], colors):
    patch.set_facecolor(color)

    
    
# add x-tick labels
plt.setp(axes[0], xticks=[y+1 for y in range(3)],
         xticklabels=['CHO (g)', 'Fat (g)', 'Protein (g)'])

fig.subplots_adjust(wspace=0.5)

plt.show()

print "The meal Carbs mean is:", np.mean(df_P1_meals_w_deletion['CHO (g)']), "grams with a std of:", np.std(df_P1_meals_w_deletion['CHO (g)']), "grams."
print "The meal Fat mean is:", np.mean(df_P1_meals_w_deletion['Fat (g)']), "grams with a std of:", np.std(df_P1_meals_w_deletion['Fat (g)']), "grams."
print "The meal Protein mean is:", np.mean(df_P1_meals_w_deletion['Protein (g)']), "grams with a std of:", np.std(df_P1_meals_w_deletion['Protein (g)']), "grams."

# Interactive plot of final meal recordings & increase points & peak points

In [ ]:
# output to static HTML file
output_file("Data_Cleaned.html")

plot = figure(plot_width=1250, plot_height=600, title="Glucose Data Cleaned Interactive Plot",
              tools='pan,wheel_zoom,reset,box_zoom,save, undo', x_axis_type='datetime')

plot.line(df_P1.Date, df_P1['mg/dL'],  line_width=2, line_alpha=0.6)
plot.square(df_P1.Date, df_P1['mg/dL'], fill_color=None, line_color="grey")

good_readings = ColumnDataSource(dict(x=df_P1_meals_w_deletion.Date, y=df_P1_meals_w_deletion['mg/dL'], z=df_P1_meals_w_deletion['CHO (g)'], k = df_P1_meals_w_deletion['Fat (g)'], d = df_P1_meals_w_deletion['Protein (g)']))
glyph_1 = Asterisk(x="x", y="y", line_color="#f0027f", fill_color=None, line_width=8)
g1 = plot.add_glyph(good_readings, glyph_1)
plot.asterisk(legend = 'Meal Recordings', line_color="#f0027f", fill_color=None, line_width=8)

plot.asterisk(x=surrounding_lowest_points_df.Date, y=surrounding_lowest_points_df['mg/dL'], legend = 'Increasing Points', line_color="#32CD32", fill_color=None, line_width=8)   

plot.asterisk(x=peak_points_df.Date, y=peak_points_df['mg/dL'], legend = 'Peaks', line_color="#0404B4", fill_color=None, line_width=8)


plot.add_tools(HoverTool(renderers=[g1], tooltips= [
    ('Glucose (mg/dL)', '@y'),
    ('CHO (g)', '@z'),
    ('Fat (g)', '@k'),
    ('Protein (g)', '@d')
]))

plot.xaxis.axis_label = "Date & Time"
plot.yaxis.axis_label = "Glucose Reading (mg/dL)"

show(plot) 

The current main dataframes are:

1. df_P1: it contains all patient data points as they were first imported from csv file

2. df_P1_meals_w_deletion: it contains all the recorded meals after removing the abnormal ones

3. surrounding_lowest_points_df: it contains the appropriate lowest points (points of increase) surrounding the recorded meal times.

4. peak_points_df: it contains the glucose peaks following the recorded meals

In [ ]:
df_P1_all_clean_1day = copy.deepcopy(df_P1_meals_w_deletion)
df_1day_increasePoints = copy.deepcopy(surrounding_lowest_points_df)
df_1day_peakpoints = copy.deepcopy(peak_points_df)

d = datetime.datetime.today()
for i in range(len(df_P1_all_clean_1day)):
    dt = df_P1_all_clean_1day.Date[i].replace(year=d.year, month=d.month, day=d.day)    
    df_P1_all_clean_1day.Date[i] = dt

fig = plt.figure(figsize= (15,15))
plt.subplots_adjust(hspace=0.5)

sub1 = fig.add_subplot(311)
sub1.set_xlabel('Hour of the Day')
sub1.set_ylabel('Glucose Reading (mg/dl)')
sub1.set_title('Meal Time Glucose Recordings')
plt.plot_date(df_P1_all_clean_1day.Date, df_P1_all_clean_1day['mg/dL'], 'ro')    

#############
#Plotting all increase points in one day
# for i in range(len(df_1day_increasePoints)):
#     dt = df_1day_increasePoints.Date[i].replace(year=d.year, month=d.month, day=d.day)    
#     df_1day_increasePoints.Date[i] = dt    
    
# sub2 = fig.add_subplot(312)
# sub2.set_xlabel('Hour of the Day')
# sub2.set_ylabel('Glucose Reading (mg/dl)')
# sub2.set_title('Increase Points nearby meals')
# plt.plot_date(df_1day_increasePoints.Date, df_1day_increasePoints['mg/dL'], 'bo')

#############
#Plotting all peak points in one day
# for i in range(len(df_1day_peakpoints)):
#     dt = df_1day_peakpoints.Date[i].replace(year=d.year, month=d.month, day=d.day)    
#     df_1day_peakpoints.Date[i] = dt    
    
# sub3 = fig.add_subplot(313)
# sub3.set_xlabel('Hour of the Day')
# sub3.set_ylabel('Glucose Reading (mg/dl)')
# sub3.set_title('Increase Points nearby meals')
# plt.plot_date(df_1day_peakpoints.Date, df_1day_peakpoints['mg/dL'], 'go')


print "The glucose mean of the recorded meals is:", np.mean(df_P1_meals_w_deletion['mg/dL']), "mg/dL with a std of:", np.std(df_P1_meals_w_deletion['mg/dL'])

plt.show()